In [1]:
%load_ext autoreload
%autoreload 2

In [10]:
import sys
from pathlib import Path

# 현재 작업 디렉토리 기준으로 프로젝트 루트 찾기
PROJECT_ROOT = Path().resolve().parent

print("PROJECT_ROOT:", PROJECT_ROOT)

if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print("sys.path[0]:", sys.path[0])

PROJECT_ROOT: /Users/anhyebin/Documents/SKN21/SKN21-3rd-4Team
sys.path[0]: /Users/anhyebin/Documents/SKN21/SKN21-3rd-4Team


In [11]:
import os
import sys
import requests
from pathlib import Path
from qdrant_client import QdrantClient
from src.utils.config import ConfigDB, ConfigAPI
from huggingface_hub import HfApi

# 절대 경로 파일 추가
root_path = str(Path(os.getcwd()).parent.absolute())
print(root_path)
if root_path not in sys.path:
    sys.path.append(root_path)

/Users/anhyebin/Documents/SKN21/SKN21-3rd-4Team


In [12]:
###
# 스냅샷 생성 후 hugging face에 자동올리기 (다운로드 받은 파일은 다시 제거함)
###

# 스냅샷 생성
snapshot = QdrantClient(host=ConfigDB.HOST, port=ConfigDB.PORT).create_snapshot(collection_name=ConfigDB.COLLECTION_NAME)
snapshot_name = snapshot.name
print(snapshot_name)

# snapshots 폴더 생성
Path("./snapshots").mkdir(parents=True, exist_ok=True)

# 스냅샷 파일 다운로드 (서버 -> 로컬)
download_url = f"http://{ConfigDB.HOST}:{ConfigDB.PORT}/collections/{ConfigDB.COLLECTION_NAME}/snapshots/{snapshot_name}"
local_file_path = "./snapshots/" + snapshot_name  # 현재 디렉토리에 저장

print(f"Downloading snapshot {snapshot_name} from {download_url}...")

with requests.get(download_url, stream=True) as r:
    r.raise_for_status()
    with open(local_file_path, 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)

print(f"Downloaded to local: {os.path.abspath(local_file_path)}")

# Hugging Face 업로드
api = HfApi(token=ConfigAPI.HUGGINGFACE_API_KEY)
repo_id = "lucymoon/skn21_3rd_4team"
path_in_repo = snapshot_name

print(f"Uploading {local_file_path} to {repo_id}...")

try:
    api.upload_file(
        path_or_fileobj=local_file_path,
        path_in_repo=path_in_repo,
        repo_id=repo_id,
        repo_type="dataset"
    )
    print("Upload completed successfully!")
    
    # 업로드 후 로컬 파일 삭제
    os.remove(local_file_path)
    print("Local snapshot file removed.")

except Exception as e:
    print(f"Upload failed: {e}")

learning_ai-5440725894880113-2026-01-05-03-40-11.snapshot
Downloaded to local: /Users/anhyebin/Documents/SKN21/SKN21-3rd-4Team/data/snapshots/learning_ai-5440725894880113-2026-01-05-03-40-11.snapshot
Uploading ./snapshots/learning_ai-5440725894880113-2026-01-05-03-40-11.snapshot to lucymoon/skn21_3rd_4team...
Upload failed: (Request ID: Root=1-695b329c-7ac4b1950dd099917e4e1e2c;341b2f67-929d-4b39-9945-2204579b1bf6)

403 Forbidden: Forbidden: pass `create_pr=1` as a query parameter to create a Pull Request.
Cannot access content at: https://huggingface.co/api/datasets/lucymoon/skn21_3rd_4team/commit/main.
Make sure your token has the correct permissions.


In [13]:
snapshot_name

'learning_ai-5440725894880113-2026-01-05-03-40-11.snapshot'

In [14]:
###
# hugging face에서 snapshot 복구하기
###

# Hugging Face Snapshot URL (Raw/Resolve URL 사용)
# blob -> resolve 로 변경하여 직접 다운로드 가능한 링크로 설정
file_name = "learning_ai-2263345336731768-2026-01-04-15-15-01.snapshot"
snapshot_url = "https://huggingface.co/datasets/lucymoon/skn21_3rd_4team/resolve/main/" + file_name

print(f"URL: {snapshot_url}")

# Qdrant 클라이언트 연결
client = QdrantClient(host="localhost", port=6333)
collection_name = ConfigDB.COLLECTION_NAME

# 기존 컬렉션이 있으면 삭제
if client.collection_exists(collection_name=collection_name):
    print(f"Removing existing collection: {collection_name}")
    client.delete_collection(collection_name=collection_name)

# URL을 통해 스냅샷 복구
client.recover_snapshot(
    collection_name=collection_name,
    location=snapshot_url,
    wait=True
)
print("Snapshot recovery from Hugging Face completed successfully!")

URL: https://huggingface.co/datasets/lucymoon/skn21_3rd_4team/resolve/main/learning_ai-2263345336731768-2026-01-04-15-15-01.snapshot
Removing existing collection: learning_ai
Snapshot recovery from Hugging Face completed successfully!


In [15]:
client.get_collections()

CollectionsResponse(collections=[CollectionDescription(name='test_collection'), CollectionDescription(name='olympic_info'), CollectionDescription(name='example'), CollectionDescription(name='sf_books'), CollectionDescription(name='sf_book_sparse'), CollectionDescription(name='olympic_info_wiki'), CollectionDescription(name='restaurant_menu'), CollectionDescription(name='sf_book_hybrid'), CollectionDescription(name='learning_ai')])